In [ ]:
!export LC_ALL=en_US.UTF-8
!export LC_CTYPE=en_US.UTF-8

In [ ]:
!pip install google_play_scraper
!pip install transformers --no-cache-dir
!pip install --upgrade transformers
!pip uninstall sentencepiece -y
!pip install sentencepiece --no-cache-dir
!pip install pydrive
!python -m spacy download fr_core_news_sm
!pip install pyspellchecker
!pip install language-tool-python
!pip install autocorrect
#!pip list --outdated
!pip install sacremoses
!pip install nltk


In [ ]:
from google_play_scraper import Sort, reviews_all
from multiprocessing import Pool
import csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from transformers import pipeline, TranslationPipeline, AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
import re
import time
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import spacy
import language_tool_python

In [ ]:

import nltk
from nltk.tokenize import word_tokenize
from autocorrect import Speller
from textblob import TextBlob
from language_tool_python import LanguageTool
import time
nltk.download('punkt')

# Instancier les correcteurs
spell_checker = Speller(lang='fr')
blob = TextBlob('fr')
tool = LanguageTool('fr')

# La phrase à tester
sentence = "Tout ait très bien me protège jamais eu de problèmes, tous anticouvid, tracing, à"

# Tokeniser la phrase
tokens = word_tokenize(sentence)

# Correction orthographique
start_time = time.time()
corrected_spell = [spell_checker(token) for token in tokens]
end_time = time.time()
print("Correction orthographique: ", " ".join(corrected_spell))
print(f"Temps de correction (SpellChecker): {end_time - start_time:.4f} secondes\n")

# Correction grammaticale
start_time = time.time()
blob = TextBlob(sentence)
corrected_grammar = str(blob.correct())
end_time = time.time()
print("Correction grammaticale: ", corrected_grammar)
print(f"Temps de correction (TextBlob): {end_time - start_time:.4f} secondes\n")

# Correction de style
start_time = time.time()
corrected_style = tool.correct(sentence)
end_time = time.time()
print("Correction de style: ", corrected_style)
print(f"Temps de correction (LanguageTool): {end_time - start_time:.4f} secondes\n")



In [ ]:
# Identifiant de l'application sur le Google Play Store
app_id = 'fr.gouv.android.stopcovid'

# Extraire tous les avis de l'application
reviews = reviews_all(
    app_id=app_id,
    lang='fr',  # Langue des avis
    sort=Sort.NEWEST,  # Ordonner les avis par date de publication (plus récents d'abord)
)

print (reviews[0:10])

[{'reviewId': '02ff027b-65c1-453a-9d01-00ebfceace17', 'userName': 'blondy pop', 'userImage': 'https://play-lh.googleusercontent.com/a-/ACB-R5SBhQoIdIYwumonJxeIi31XZ3V3zeiQAtWTHK_G', 'content': "En théorie c'est pas mal mais c'est super malin de pas avoir mis de système de sauvegarde des document", 'score': 1, 'thumbsUpCount': 0, 'reviewCreatedVersion': '5.5.0', 'at': datetime.datetime(2023, 3, 16, 8, 50, 15), 'replyContent': None, 'repliedAt': None}, {'reviewId': '0c06255a-a53a-458c-a2f5-72ee82f2d4a3', 'userName': 'Jean-Luc Dancoine', 'userImage': 'https://play-lh.googleusercontent.com/a-/ACB-R5TGK-P_MnSxJQxUJf7eLNLEF3aiP1vlXFt7j9h8SA', 'content': 'Très barbant,ces enquêtes, mais je le fait de bon cœur', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '5.5.0', 'at': datetime.datetime(2023, 3, 15, 18, 56, 31), 'replyContent': None, 'repliedAt': None}, {'reviewId': '6e16df96-55f1-4099-8e97-8e078a06e55e', 'userName': 'Fabrice Blay', 'userImage': 'https://play-lh.googleusercontent.

In [ ]:
# Trier les commentaires par nombre de likes
reviews_sorted = sorted(reviews, key=lambda x: x['thumbsUpCount'], reverse=True)

print(reviews_sorted[0:2000])

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
#@title Pre-process, correction, translation, export
from language_tool_python import LanguageTool
import nltk
import numpy as np
import concurrent.futures
import itertools
import torch
import pandas as pd
import os

def grouper(iterable, n):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=None)

# Load the LanguageTool model
tool = LanguageTool('fr')

# Fonction pour prétraiter et corriger les commentaires avant la traduction
def preprocess_batch(comments):
    # Remove HTML tags
    comments = [re.sub('<[^<]+?>', '', comment) for comment in comments]
    # Remove links
    comments = [re.sub(r'http\S+', '', comment) for comment in comments]
    # Remove useless special characters (emojis, etc.)
    comments = [re.sub('[^A-Za-z0-9\s\.\'\?,!]+', '', comment) for comment in comments]
    # Normalize text
    comments = [re.sub(r"(\w+)'\s*(\w+)", r"\1\2", comment) for comment in comments]
    # Convert to lowercase
    # comments = [''.join(c.lower() for c in comment) for comment in comments]
    # Add spaces after punctuation marks
    comments = [re.sub(r"([a-zA-Z])([.,;:!?)])", r"\1 \2", comment) for comment in comments] 
    
    # Correct spelling and grammar
    def correct_batch(batch):
        return [tool.correct(comment) for comment in batch]

    with ThreadPoolExecutor() as executor:
        corrected_comments = []
        batches = grouper(comments, batch_size)
        futures = []
        for batch in batches:
            batch = [comment for comment in batch if comment is not None]
            futures.append(executor.submit(correct_batch, batch))
        for future in as_completed(futures):
            corrected_comments.extend(future.result())

    corrected_comments_filtered = []
    for i in range(len(corrected_comments)):
        corrected = corrected_comments[i]
        if corrected and len(corrected) >= 10 and len(corrected) <= 5000:
            corrected_comments_filtered.append(corrected)
        else:
            corrected_comments_filtered.append("")
    return corrected_comments_filtered


# load the model and tokenizer for translation 
model_name = "Helsinki-NLP/opus-mt-ROMANCE-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to('cuda')

#Set up the translation pipeline
translator = TranslationPipeline(model=model, tokenizer=tokenizer, task="translation_fr_to_en")

# Translate the comments in parallel using threads and batches
def translate_batch(batch):
    with torch.no_grad():  # we don't need to compute gradients during inference
        input_ids = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, truncation=True)['input_ids'].to('cuda')
        outputs = model.generate(input_ids)
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return decoded_outputs

def translate_comments(preprocessed_comments, batch_size=32):
    translated_comments = []
    with ThreadPoolExecutor() as executor:
        batches = grouper(preprocessed_comments, batch_size)
        futures = []
        for batch in batches:
            batch = [comment for comment in batch if comment is not None]
            futures.append(executor.submit(translate_batch, batch))
        for future in as_completed(futures):
            translated_comments.extend(future.result())
    return translated_comments

# Select the first 20000/60000 reviews
selectedreviews = reviews_sorted[0:10000]
selectedcomments = [review['content'] for review in selectedreviews]

# Split the comments into batches
batch_size = 32
comment_batches = [selectedcomments[i:i+batch_size] for i in range(0, len(selectedcomments), batch_size)]

# Preprocess the comments before translation
preprocessed_comments = []
for review_batch in tqdm(comment_batches, desc="Preprocessing in progress"):
    batch_preprocessed = preprocess_batch(review_batch)
    preprocessed_comments.extend(batch_preprocessed)
    

# Libération de la mémoire GPU
torch.cuda.empty_cache()

def associate_comments(preprocessed_comments, translated_comments):
    translated_comments_dict = {}
    for i in range(len(preprocessed_comments)):
        preprocessed_comment = preprocessed_comments[i]
        if preprocessed_comment is not None and i < len(translated_comments):
            translated_comments_dict[preprocessed_comment] = translated_comments[i]
    return translated_comments_dict

# Split the preprocessed comments into batches
preprocessed_batches = [preprocessed_comments[i:i+batch_size] for i in range(0, len(preprocessed_comments), batch_size)]
#translate the comments
translated_comments = []
for batch in tqdm(preprocessed_batches, desc="Translation in progress"):
    batch_translations = translate_comments(batch)
    translated_comments.extend(batch_translations)

# Libération de la mémoire GPU
torch.cuda.empty_cache()
# remove punctuation for csv format and future NLP
preprocessed_comments = [re.sub(r'[^\w\s]', '', comment) for comment in preprocessed_comments]
translated_comments = [re.sub(r'[^\w\s]', '', comment) for comment in translated_comments]
print(preprocessed_comments)
print(translated_comments)
# Associate the translated comments with the original comments in a dictionary
translated_comments_dict = associate_comments(preprocessed_comments, translated_comments)
# Define a function to export the data
def export_data_to_csv(reviews, preprocessed_comments, translated_comments_dict, filename, batch_size=16):
    # Remove existing file if it exists
    if os.path.exists(filename):
        os.remove(filename)
    # Create a DataFrame with the reviews data
    df = pd.DataFrame(reviews)
    # Rename the 'content' column to 'original_comment'
    df = df.rename(columns={'content': 'original_comment'})
    # Add a column with the preprocessed comments
    df['preprocessed_comment'] = [comment for comment in preprocessed_comments]
    # Add a column with the translated comments
    df['translated_comment'] = [translated_comments_dict.get(comment, "") for comment in df['preprocessed_comment']]
    # Drop unnecessary columns
    df = df.drop(['userImage', 'reviewCreatedVersion', 'replyContent', 'repliedAt', 'original_comment'], axis=1)
    # Define a function to process each batch in parallel
    def process_batch(start, end):
        batch_df = df[start:end]
        batch_df.to_csv(filename, mode='a', index=False, header=(start == 0)) #put mode="a" and not "w" otherwise each batch of 32 will write on the other and the exported file will be size of the last batch everytime

    # Process the data in parallel using threads and batches
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(0, len(df), batch_size):
            futures.append(executor.submit(process_batch, i, i+batch_size))
        for future in concurrent.futures.as_completed(futures):
            future.result()

# Call the function to export the data
export_data_to_csv(selectedreviews, preprocessed_comments, translated_comments_dict, 'translated_reviews.csv', batch_size=32)

print("The reviews have been successfully exported to the CSV file.")

df = pd.read_csv("translated_reviews.csv")
print(df.head())

Translation in progress:   0%|          | 0/313 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Translation in progress: 100%|██████████| 313/313 [41:45<00:00,  8.00s/it]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



The reviews have been successfully exported to the CSV file.
                               reviewId          userName  score  \
0  e1aeca72-bec3-4c26-a170-9fe49b8c7d9d  Corentin Prigent      5   
1  388a0e9e-d7a3-4f19-b792-2df821b6da3f   claude leclercq      3   
2  46463794-2169-4191-b3f2-136f7ade6760     Timothée Brun      5   
3  2d4a1f11-652e-4442-8629-7f56ad511240    Olivier FEBWIN      4   
4  8eeb4d02-2b8e-422c-93d8-3ef125239e50     Baptiste Piat      4   

   thumbsUpCount                   at  \
0           3387  2022-02-13 05:56:53   
1           2511  2021-04-06 06:40:05   
2           1829  2022-01-02 08:43:48   
3           1692  2022-10-12 09:51:39   
4           1658  2022-01-28 21:54:09   

                                preprocessed_comment  \
0  App excellente pas de pub dure de vie assez lo...   
1  Premier avantage immédiatement visible plus be...   
2  Ces jeux est génial il y a plein de possibilit...   
3  Pourquoi obliger lactivation de toutes les not...   
4  

In [ ]:
print(preprocessed_comments)

['App excellente pas de pub dure de vie assez longue Le jeu est parfait pour les longs trajets dans lavion par exemple Jai été de voir la fin du jeu pour linstant je ne me suis pas faits poil et de ce que je vois sur MSN Twitter Myspace ou Blog personne na encore fini le jeu de tout faon Si vous commencez le jeu faites gaffe au niveau Delta jai eu du mal le passer mais une fois pas je trouve le jeu assez facile surtout au niveau Omicron', 'Premier avantage immédiatement visible plus besoin de remplir lattestation papier il suffit de choisir le motif de sortie Lefficacité pour lidentification des contacts reste prouver Un inconvénient notable la batterie baisse la vitesse grand V Il faut recharger le smartphone tous les jours Mme si on lutilise peu par ailleurs Donc désactiver lapplication chez soi quand on na pas de visite', 'Ces jeux est génial il y a plein de possibilité et un gameplay amusant Je suis content davoir eu ce code qui me permet de passer les portes et les portails pour r

In [ ]:
# Libération de la mémoire GPU
torch.cuda.empty_cache()

In [ ]:
# Connexion à Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Recherche du dossier dans lequel copier le fichier CSV
folder_name = 'TOUSANTICOVID'
folder_list = drive.ListFile({'q': "mimeType='application/vnd.google-apps.folder' and trashed=false and title='" + folder_name + "'"}).GetList()
if len(folder_list) > 0:
    folder = folder_list[0]
else:
    folder = drive.CreateFile({'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder'})
    folder.Upload()
    
# Upload du fichier CSV dans le dossier
file = drive.CreateFile({'title': 'translated_reviews.csv', 'parents': [{'id': folder['id']}]})
file.SetContentFile('translated_reviews.csv') #assurez-vous également que le fichier CSV sur le Drive est correctement téléchargé en utilisant la méthode SetContentFile() avant de télécharger le fichier avec la méthode Upload()
file.Upload()

# Ouvrir le fichier sur le Drive pour vérifier que les données ont bien été copiées
file = drive.CreateFile({'id': file['id']})
content = file.GetContentString()
print("Le fichier CSV sur le Drive contient:\n", content)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
